In [ ]:
# global optimization to find coefficients for weighted ensemble 

import os
import cv2
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from numpy import mean
from numpy import std
from numpy import array
from numpy import argmax
from numpy import tensordot
from numpy.linalg import norm
from scipy.optimize import differential_evolution
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model


# fit model on dataset
def fit_model(train_dir, target_x,target_y, num_epochs):
    train_datagen = ImageDataGenerator(rescale = 1.0/255.0, rotation_range = 40, zoom_range = 0.2, 
                                  shear_range = 0.2, width_shift_range = 0.2, height_shift_range = 0.2, 
                                  horizontal_flip = True, fill_mode = "nearest")
    train_generator = train_datagen.flow_from_directory(train_dir, target_size = (target_x,target_y), batch_size = 32, class_mode = "categorical")

    
    #loading pretrained model
    # Import the inception model  
    from tensorflow.keras.applications.inception_v3 import InceptionV3
    # Create an instance of the inception model from the local pre-trained weights
    local_weights_file = "Dataset/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
    pre_trained_model = InceptionV3(input_shape = (target_x,target_y,3), include_top = False, weights = None)
    pre_trained_model.load_weights(local_weights_file)
    
    # Make all the layers in the pre-trained model non-trainable
    for layer in pre_trained_model.layers:
      layer.trainable = False
    last_layer = pre_trained_model.get_layer('mixed9')
    print('last layer output shape: ', last_layer.output_shape)
    last_output = last_layer.output
    
    # define model
    # Flatten the output layer to 10 dimension
    x = layers.Flatten()(last_output)
    # Add a fully connected layer with 512 hidden units and ReLU activation
    x = layers.Dense(512, activation = 'relu')(x)
    # Add a dropout rate of 0.2  
    # Add a fully connected layer with 1,024 hidden units and ReLU activation
    x = layers.Dense(1024, activation = 'relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)                  
    # Add a final sigmoid layer for classification
    x = layers.Dense(10, activation = 'softmax')(x)           
    
    model = Model(pre_trained_model.input, x) 
    model.compile(optimizer = "adam", 
              loss = 'categorical_crossentropy', metrics = ["accuracy"])
    # fit model
    history = model.fit(train_generator,epochs= num_epochs,verbose=1)
    acc = history.history['accuracy'][-1]
    return model

#getting the testing data and the labels in an numpy array

def get_data(dirname, img_type):
    testX = []
    testy = []
    folders = os.listdir(dirname)
    for folder in folders:
    for image in os.listdir(os.path.join(base_path,folder)):
        
        img = cv2.imread(os.path.join(base_path,folder,image))
        if img_type == 'skin':
            #converting from gbr to YCbCr color space
            img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
            #skin color range forYCbCr color space 
            YCrCb_mask = cv2.inRange(img_YCrCb, (0,127,77), (240,150,120)) 
            YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
            YCrCb_result = cv2.bitwise_not(YCrCb_mask)
            resized_image = cv2.resize(YCrCb_result, (75, 75))
        else:
            resized_image = cv2.resize(img, (300, 300))
        testX.append(resized_image)
        testy.append(folder)
    return testX, testy

    

# make an ensemble prediction for multi-class classification
def ensemble_predictions(model_CNN,model_skin, weights, testX,testX_skin):
    # make predictions
    yhats = [model_CNN.predict(testX),model_skin.predict(testX_skin)]
    yhats = array(yhats)
    # weighted sum across ensemble members
    summed = tensordot(yhats, weights, axes=((0),(0)))
    # argmax across classes
    result = argmax(summed, axis=1)
    return result

# # evaluate a specific number of members in an ensemble
def evaluate_ensemble(model_CNN,model_skin, weights, testX, testy,testX_skin):
    # make prediction
    yhat = ensemble_predictions(model_CNN,model_skin, weights, testX,testX_skin)
    # calculate accuracy
    return accuracy_score(testy, yhat)

# normalize a vector to have unit norm
def normalize(weights):
    # calculate l1 vector norm
    result = norm(weights, 1)
    # check for a vector of all zeros
    if result == 0.0:
        return weights
    # return normalized vector (unit norm)
    return weights / result

# loss function for optimization process, designed to be minimized
def loss_function(weights, model_CNN,model_skin, testX, testy,testX_skin):
    # normalize weights
    normalized = normalize(weights)
    # calculate error rate
    return 1.0 - evaluate_ensemble(model_CNN,model_skin, normalized, testX, testy,testX_skin)



# get test data

testX, testy = get_data("Dataset/imgs/test","normal")
testX_skin, _ = get_data("Dataset/imgs/test_skin","skin")
testy = testy = pd.get_dummies(np.array(testy)).to_numpy()

# fit all models
model_CNN =  fit_model("Dataset/imgs/train", 300,300, 15)
model_skin =  fit_model("Dataset/skin_imgs", 75,75, 50)

# define bounds on each weight
bound_w = [(0.0, 1.0)  for _ in range(2)]
# arguments to the loss function
search_arg = (model_CNN,model_skin, testX, testy,testX_skin)


# global optimization of ensemble weights
result = differential_evolution(loss_function, bound_w, search_arg, maxiter=100, tol=1e-5)
# get the chosen weights
# the ‘x‘ key contains the optimal set of weights found during the search.
weights = normalize(result['x'])

print('Optimized Weights: %s' % weights)
# evaluate chosen weights
score = evaluate_ensemble(model_CNN,model_skin, weights, testX, testy)

print('Optimized Weights Score: %.3f' % score)

TypeError: VideoCapture.open() missing required argument 'index' (pos 1)

[]

False

Error opening video stream or file


False